# Training a Simple LUTNN

In [ ]:
import torch
import torch.optim as optim
import sys
from pathlib import Path
from lutnn.lutlayer import LUTLayer, Aggregation

In [11]:
from utils.uci_datasets import BreastCancerDataset

train_set = BreastCancerDataset('./data-uci', split='train', download=True, with_val=False)
test_set = BreastCancerDataset('./data-uci', split='test', with_val=False)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=int(1e6), shuffle=False)
input_dim_dataset = 51
num_classes = 2

In [ ]:
PROJECT_ROOT = Path.cwd().parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from hdl.convert2vhdl import get_model_params, gen_vhdl_code
from hdl.convert2sv import gen_sv_code

In [13]:
# Define a simple LUTNN
class SimpleLUTNN(torch.nn.Module):
    def __init__(self):
        super(SimpleLUTNN, self).__init__()
        self.layer1 = LUTLayer(input_dim=51, lut_size=4, n_luts=512)
        self.layer2 = LUTLayer(input_dim=512, lut_size=4, n_luts=1024)
        self.layer3 = Aggregation(num_classes=2, tau=10.)

    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        return x

In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = SimpleLUTNN().to(device)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

cpu


In [15]:
# Training loop
num_epochs = 4
for epoch in range(num_epochs):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, target)
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{batch_idx}/{len(train_loader)}], Loss: {loss.item():.4f}')

# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f'Test Accuracy: {100 * correct / total:.2f}%')

Epoch [1/4], Step [0/2], Loss: 0.7340
Epoch [2/4], Step [0/2], Loss: 0.5976
Epoch [3/4], Step [0/2], Loss: 0.5207
Epoch [4/4], Step [0/2], Loss: 0.5116
Test Accuracy: 64.29%


In [16]:
# Wrap the layers so the HDL export functions can find them
model.model = torch.nn.Sequential(
    torch.nn.Flatten(),
    model.layer1,
    model.layer2,
    model.layer3
)

number_of_layers, num_neurons, lut_sizes, number_of_inputs, number_of_classes = get_model_params(model)
model_name = 'breast_lutnn'

In [17]:
# VHDL generation
gen_vhdl_code(model, model_name, number_of_layers, number_of_classes, number_of_inputs, num_neurons, lut_sizes)
print(f'VHDL code generated in data/VHDL/{model_name}/')

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): LUTLayer(num_luts=512, lut_size=4)
  (2): LUTLayer(num_luts=1024, lut_size=4)
  (3): Aggregation(num_classes=2, tau=10.0)
)
tensor([[1, 1, 0,  ..., 0, 0, 1],
        [1, 1, 0,  ..., 0, 1, 0],
        [1, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 1, 1, 0]])
tensor([[1, 0, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 1, 0, 1],
        [1, 0, 1,  ..., 1, 1, 0],
        ...,
        [1, 0, 1,  ..., 0, 1, 1],
        [0, 1, 0,  ..., 0, 0, 1],
        [0, 1, 0,  ..., 0, 0, 1]])
VHDL code generated in data/VHDL/breast_lutnn/


In [18]:
# SV generation
gen_sv_code(model, model_name, number_of_layers, number_of_classes, number_of_inputs, num_neurons, lut_sizes)
print(f'SystemVerilog code generated in data/sv/{model_name}/')

Sequential(
  (0): Flatten(start_dim=1, end_dim=-1)
  (1): LUTLayer(num_luts=512, lut_size=4)
  (2): LUTLayer(num_luts=1024, lut_size=4)
  (3): Aggregation(num_classes=2, tau=10.0)
)
tensor([[1, 1, 0,  ..., 0, 0, 1],
        [1, 1, 0,  ..., 0, 1, 0],
        [1, 0, 0,  ..., 1, 1, 1],
        ...,
        [0, 1, 1,  ..., 0, 0, 0],
        [0, 1, 1,  ..., 0, 0, 0],
        [1, 1, 0,  ..., 1, 1, 0]])
tensor([[1, 0, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 1, 0, 1],
        [1, 0, 1,  ..., 1, 1, 0],
        ...,
        [1, 0, 1,  ..., 0, 1, 1],
        [0, 1, 0,  ..., 0, 0, 1],
        [0, 1, 0,  ..., 0, 0, 1]])
SystemVerilog code generated in data/sv/breast_lutnn/
